Integrantes: 

Camilo Andrés López Sarmiento      201511535
          
Juan Sebastian Sepulveda Pacheco   201621805

In [ ]:
# Importación de librerias
import re #Operaciones con expresiones regulares
import tweepy #es el cliente de Python para la API oficial de Twitter 
from tweepy import OAuthHandler #Credenciales dadas para el uso de la API
from textblob import TextBlob #textblob es la biblioteca de Python para procesar datos textuales.
import pandas as pd

**import re** (Breve Explicación)

Las expresiones regulares usan el carácter de barra inversa ('\') para indicar formas especiales o para permitir el uso de caracteres especiales sin invocar su significado especial. Esto choca con el uso de Python de este carácter para el mismo propósito con los literales de cadena; por ejemplo, para hacer coincidir una barra inversa literal, se podría escribir '\\\\' como patrón, porque la expresión regular debe ser \\, y cada barra inversa debe ser expresada como \\ dentro de un literal de cadena regular de Python.

In [ ]:
#Clase genérica de Twitter para análisis de sentimiento.
class TwitterClient(object):
     
    #Constructor de clase o método de inicialización.
    def __init__(self):
    
        #Claves y tokens de la Consola de desarrollo de Twitter
        consumer_key = '4ToNBia2fVMTklK4YRPG7CGEF'
        consumer_secret = '3LT553VmcrPOmkOD6VbIZtYcAs0pPhVNtYgD6lOjRjO1wiOTK2'
        access_token = '885375894-n41ewfxTk90YSreuAye6llwvOOuht96a5UBvmIIC'
        access_token_secret = '2dnSZo1XrdX0jj97Q6UyjpYLwTLG91WotWWDrsD579wfo'
  
        #Intento de autenticación
        try:
            #Crear objeto OAuthHandler (OAuth authentication handler)
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            #Establecer token de acceso y secreto
            self.auth.set_access_token(access_token, access_token_secret)
            #Crear un objeto API tweepy para obtener tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Autenticación fallida")
  

    #Función de utilidad para limpiar el texto del tweet eliminando enlaces,
    #caracteres especiales usando declaraciones regulares simples
    #y elimina caracteres chinos (algunos)  
    def clean_tweet(self, tweet):

        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a)|(\w+:\/\/\S+)", " ", tweet).split())
  
    #Función de utilidad para clasificar el sentimiento del tweet 
    #pasado usando el método de sentimiento de textblob
    def get_tweet_sentiment(self, tweet):

        #Crear el objeto TextBlob del texto del tweet pasado
        analysis = TextBlob(self.clean_tweet(tweet))

        #Establecer sentimiento
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
    #Función principal para buscar tweets y analizarlos
    def get_tweets(self, query, count = 10):

        # lista vacía para almacenar tweets analizados
        tweets = []
  
        try:
            # llame a la API de Twitter para obtener tweets
            fetched_tweets = self.api.search(q = query, count = count)
  
            # analizar tweets uno por uno
            for tweet in fetched_tweets:
                # diccionario vacío para almacenar los parámetros requeridos de un tweet
                parsed_tweet = {}
  
                # guardando el texto del tweet
                parsed_tweet['text'] = tweet.text
                # guardando el sentimiento del tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # agregar tweet analizado a la lista de tweets
                if tweet.retweet_count > 0:
                    # si el tweet tiene retweets, asegúrese de que se agregue solo una vez
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # devolver tweets analizados
            return tweets
  
        except tweepy.TweepError as e:
            # error de impresión (si lo hay)
            print("Error : " + str(e))

In [ ]:
import pandas as pd
def main():
    Positivos_T=[]
    Negativos_T=[]
    Neutros_T=[]
    # creando objeto de la clase TwitterClient 
    api = TwitterClient()

    # función de llamada para obtener tweets
    tweets = api.get_tweets(query = 'ach', count = 500)
   
    # elegir tweets positivos de tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']

    # porcentaje de tuits positivos
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))

    # seleccionando tweets negativos de tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

     # seleccionando tweets Neutros de tweets
    nntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']

    # porcentaje de tweets negativos
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))

    # porcentaje de tuits neutrales
    print("Neutral tweets percentage: {} % \
        ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
    #Creamos el DataFrame
    df = pd.DataFrame()

    # imprimir los tweets positivos
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        Positivos_T.append(tweet['text'])
    df = df.append(Positivos_T) 
    print(df)
  
    # imprimir los  tweets negativos
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        Negativos_T.append(tweet['text'])
    df = df.append(Negativos_T) 
    print(df)
    
    # imprimir los tweets negativos
    print("\n\nNeutral tweets:")
    for tweet in nntweets[:10]:
        Neutros_T.append(tweet['text'])
    df = df.append(Neutros_T) 
    print(df)

if __name__ == "__main__":
    # llamando a la función principal
    main()

Positive tweets percentage: 4.081632653061225 %
Negative tweets percentage: 4.081632653061225 %
Neutral tweets percentage: 91.83673469387755 %         


Positive tweets:
                                                   0
0      @ach_kanchhu Happy Birthday in advance maya 🎂
1  RT @NorikStormblaze: @madzman12 @younwimMn One...
2  RT @younwimMn: Hot take:\n\nWell choreographed...
3  @_Katja_Diehl_ Und ich nenne Pleiten-Christian...


Negative tweets:
                                                   0
0      @ach_kanchhu Happy Birthday in advance maya 🎂
1  RT @NorikStormblaze: @madzman12 @younwimMn One...
2  RT @younwimMn: Hot take:\n\nWell choreographed...
3  @_Katja_Diehl_ Und ich nenne Pleiten-Christian...
0  Hi @reliancejio, I am rural area hardly 8-10km...
1  Nailed it 35% profit in less than 3days from s...
2  @TMotzarella Ach scheiße 😩 wie unfair einfach!...
3  @carogranny ach, ich trag seit 40 Jahren welch...


Neutral tweets:
                                                  